<a href="https://colab.research.google.com/github/DaraRahma536/TensorFlow-in-Action/blob/main/Chapter_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chapter 7: Teaching Machines to See Better: Improving CNNs and Making Them Confess**

# **1. Teknik untuk Mengurangi Overfitting**
---
Overfitting terjadi ketika model belajar terlalu baik dari data training hingga menangkap noise dan pola acak, sehingga performa buruk pada data baru. Teknik untuk mengatasinya:

* **Data Augmentation:** Menciptakan variasi data training melalui transformasi gambar (rotasi, translasi, brightness, dll.) untuk membuat model lebih robust.
* **Dropout:** Mematikan neuron secara acak selama training, memaksa model belajar representasi yang redundan.
* **Early Stopping:** Menghentikan training ketika performa validation tidak membaik lagi.

Implementasi Kode:

In [ ]:
# 7.1.1 Data Augmentation dengan ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Generator dengan augmentasi untuk training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generator tanpa augmentasi untuk validation
val_datagen = ImageDataGenerator(rescale=1./255)

# 7.1.2 Dropout Implementation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential

model = Sequential([
    # ... layers sebelumnya
    Dropout(0.5),  # Dropout 50%
    # ... layers selanjutnya
])

# 7.1.3 Early Stopping Callback
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',    # Monitor validation loss
    patience=10,           # Tunggu 10 epoch tanpa improvement
    restore_best_weights=True  # Kembali ke weights terbaik
)

# Training dengan callback
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[early_stopping]
)

**Penjelasan Detail:**
* **rotation_range=40:** Gambar dirotasi secara acak hingga 40 derajat
* **width_shift_range=0.2:** Geser gambar hingga 20% lebarnya
* **Dropout(0.5):** 50% neuron dimatikan secara acak setiap batch
* **EarlyStopping:** Training berhenti jika val_loss tidak membaik selama 10 epoch

# **2. Menuju Minimalisme: Minception sebagai Pengganti Inception**
---
Minception adalah varian ringan dari Inception-ResNet-v2 dengan:

* **Batch Normalization:** Normalisasi output tiap layer untuk stabilisasi training
* **Residual Connections:** "Shortcut" yang menghubungkan input ke output layer
* **Inception Blocks:** Multiple convolution paths dengan kernel size berbeda

Implementasi Komponen Minception:

In [ ]:
# 7.2.1 Stem of Minception
def stem(inp, activation='relu', bn=True):
    conv1 = Conv2D(32, (3,3), strides=(2,2), padding='same')(inp)
    if bn:
        conv1 = BatchNormalization()(conv1)
    conv1 = Activation(activation)(conv1)
    # ... lebih banyak layers
    return output

# 7.2.2 Inception-ResNet Block Type A
def inception_resnet_a(inp, n_filters, activation='relu', bn=True):
    # Branch 1: 1x1 convolution
    branch1 = Conv2D(n_filters[0], (1,1), padding='same')(inp)
    if bn: branch1 = BatchNormalization()(branch1)

    # Branch 2: 1x1 → 3x3 → 3x3
    branch2 = Conv2D(n_filters[1], (1,1), padding='same')(inp)
    branch2 = Conv2D(n_filters[2], (3,3), padding='same')(branch2)
    branch2 = Conv2D(n_filters[3], (3,3), padding='same')(branch2)
    if bn: branch2 = BatchNormalization()(branch2)

    # Concatenate + Residual Connection
    concat = Concatenate()([branch1, branch2])
    output = Conv2D(n_filters[4], (1,1), padding='same')(concat)

    # Residual Connection
    if inp.shape[-1] == output.shape[-1]:
        output = Add()([output, inp])

    return Activation(activation)(output)

# 7.2.5 Full Minception Model
def build_minception(input_shape=(64,64,3), num_classes=200):
    inputs = Input(shape=input_shape)

    # Random cropping dan contrast adjustment
    from tensorflow.keras.layers.experimental.preprocessing import RandomCrop, RandomContrast
    x = RandomCrop(56, 56)(inputs)
    x = RandomContrast(0.3)(x)

    # Stem
    x = stem(x)

    # Inception-ResNet Blocks
    x = inception_resnet_a(x, [32, 32, 48, 64, 384])
    x = reduction_block(x, [256, 256, 384, 384])
    x = inception_resnet_b(x, [192, 128, 160, 192, 1152])
    x = inception_resnet_b(x, [192, 128, 160, 192, 1152])

    # Final Layers
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs)

**Penjelasan Komponen:**
* **BatchNorm:** Normalisasi → Activation → Conv (urutan yang umum)
* **Residual Connection:** output = Add()([output, input])
* **Factorized Convolutions:** 7×7 conv dipecah menjadi 1×7 dan 7×1

# **3. Transfer Learning dengan Pretrained Networks**
---
Transfer learning menggunakan model yang sudah dilatih pada dataset besar (ImageNet) dan melakukan fine-tuning untuk task spesifik.

Keuntungan:
* Butuh data lebih sedikit
* Training lebih cepat
* Performa umumnya lebih baik

Implementasi:

In [ ]:
# 7.3.1 Menggunakan Pretrained Inception-ResNet-v2
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

# Load pretrained model tanpa top layer
base_model = InceptionResNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3),
    pooling='avg'
)

# Freeze base model (optional)
base_model.trainable = False

# Bangun model lengkap
model = Sequential([
    Input(shape=(224, 224, 3)),
    base_model,
    Dropout(0.4),
    Dense(200, activation='softmax')  # 200 classes untuk tiny-ImageNet
])

# Compile dengan learning rate kecil
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 7.3.2 Fine-tuning Strategy
def fine_tune_model(model, base_model, trainable_layers=100):
    # Unfreeze beberapa layer terakhir
    base_model.trainable = True

    # Freeze semua layer kecuali N terakhir
    for layer in base_model.layers[:-trainable_layers]:
        layer.trainable = False

    # Recompile dengan learning rate lebih kecil
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Learning Rate Scheduling
from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,      # Kurangi LR 10x
    patience=5,      # Tunggu 5 epoch
    min_lr=1e-7      # LR minimum
)

**Strategi Fine-tuning:**
* Freeze semua layer, train hanya classifier baru
* Unfreeze beberapa layer terakhir, train bersama classifier
* Learning rate lebih kecil untuk fine-tuning

# **4. Grad-CAM: Interpretasi Model CNN**
---
Grad-CAM (Gradient-weighted Class Activation Mapping) memvisualisasikan area mana dalam gambar yang paling berpengaruh terhadap prediksi model.

**Prinsip Kerja:**
* Hitung gradient output class terhadap feature maps layer convolutional terakhir
* Average pooling gradient spatial untuk mendapatkan weights tiap channel
* Kombinasikan weighted feature maps
* Apply ReLU untuk highlight area positif

Implementasi:

In [ ]:
# 7.4.1 Implementasi Grad-CAM
import tensorflow as tf
import numpy as np
import cv2

class GradCAM:
    def __init__(self, model, layer_name):
        self.model = model
        self.layer_name = layer_name
        self.grad_model = tf.keras.models.Model(
            inputs=[model.inputs],
            outputs=[model.get_layer(layer_name).output, model.output]
        )

    def compute_heatmap(self, image, class_idx):
        with tf.GradientTape() as tape:
            conv_outputs, predictions = self.grad_model(image)
            loss = predictions[:, class_idx]

        # Compute gradients
        grads = tape.gradient(loss, conv_outputs)

        # Pool gradients spatially
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

        # Weight feature maps by gradients
        conv_outputs = conv_outputs[0]
        heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)

        # ReLU and normalization
        heatmap = tf.maximum(heatmap, 0)
        heatmap /= tf.reduce_max(heatmap)

        return heatmap.numpy()

    def overlay_heatmap(self, image, heatmap, alpha=0.4):
        # Resize heatmap to match image
        heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
        heatmap = np.uint8(255 * heatmap)

        # Apply colormap
        heatmap_colored = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

        # Superimpose
        superimposed = cv2.addWeighted(image, 1-alpha, heatmap_colored, alpha, 0)

        return superimposed

# 7.4.2 Contoh Penggunaan
def visualize_gradcam(model, image_path, layer_name='conv5_block3_out'):
    # Load dan preprocess image
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.inception_resnet_v2.preprocess_input(img_array)

    # Predict class
    preds = model.predict(img_array)
    pred_class = np.argmax(preds[0])

    # Compute heatmap
    gradcam = GradCAM(model, layer_name)
    heatmap = gradcam.compute_heatmap(img_array, pred_class)

    # Visualize
    original_img = cv2.imread(image_path)
    original_img = cv2.resize(original_img, (224, 224))

    superimposed = gradcam.overlay_heatmap(original_img, heatmap)

    return original_img, superimposed, pred_class, preds[0][pred_class]

**Interpretasi Heatmap:**
* Area merah: High activation, area kritis untuk prediksi
* Area biru: Low activation, kurang relevan
* Validasi: Cek apakah model fokus pada objek yang benar